# IndoXTC - Fine-tune Head Prosa [XLM-R] [ALL]
Exploring Indonesian hate speech/abusive & sentiment text classification using multilingual language model.

This kernel is a part of my undergraduate final year project.
Checkout the full github repository:
https://github.com/ilhamfp/indonesian-text-classification-multilingual

In [1]:
#############################
# Experiment configurations #
#############################

INDO_DATA_NAME = 'prosa'
DATA_PATH_INDO = '../input/indoxtc-extracting-prosa-features-mbert'

FOREIGN_DATA_NAME = 'yelp'
DATA_PATH_FOREIGN = '../input/indoxtc-combining-yelp-features-mbert'

MODEL_NAME = 'mBERT'

EXPERIMENT_TYPE_LIST = ['A', 'B', 'C'] # A / B / C
TOTAL_DATA_LIST = [500, 1000, 2500, 5000, 7500, 10981]
FOREIGN_LANG_DATA_MULT_LIST = [0.25, 0.5, 0.75, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10]
RANDOM_SEED_LIST = [4,5,6]
VALIDATION_DATA = 0.1

In [2]:
import os
import numpy as np
import pandas as pd
from load_data import load_experiment_features, set_random_seed_data
from model_head import set_seed, train, test, evaluate

CUDA is not available.  Training on CPU ...


## Experiment

In [3]:
result_list_final = []
for RANDOM_SEED in RANDOM_SEED_LIST:
    print("\n############################")
    print("### START RANDOM_SEED {} ###".format(RANDOM_SEED))
    print("############################")
    result_list = []
    for TOTAL_DATA in TOTAL_DATA_LIST:
        print("\n##### TOTAL_DATA {} #####".format(TOTAL_DATA))

        for EXPERIMENT_TYPE in EXPERIMENT_TYPE_LIST:
            for FOREIGN_LANG_DATA_MULT in FOREIGN_LANG_DATA_MULT_LIST:
                set_seed(seed=RANDOM_SEED)
                set_random_seed_data(seed=RANDOM_SEED)
                if EXPERIMENT_TYPE != 'C':
                    print('\n~~Result_{}_{}~~'.format(EXPERIMENT_TYPE,
                                                       TOTAL_DATA))
                else:
                    print('\n~~Result_{}_{}_{}~~'.format(EXPERIMENT_TYPE,
                                                   TOTAL_DATA,
                                                   FOREIGN_LANG_DATA_MULT))

                train_loader, valid_loader, test_loader = load_experiment_features(DATA_PATH_INDO,
                                                                                   DATA_PATH_FOREIGN,
                                                                                   tipe=EXPERIMENT_TYPE, 
                                                                                   total_data=TOTAL_DATA, 
                                                                                   foreign_mult=FOREIGN_LANG_DATA_MULT,
                                                                                   valid_size=VALIDATION_DATA)

                train(train_loader, valid_loader, learning_rate=0.0001, input_dim=768)
                print("## Test phase...")
                y_true, y_pred_proba = test(test_loader, input_dim=768)
                max_f1, max_recall, max_precision, max_threshold = evaluate(y_true, 
                                                                            y_pred_proba,
                                                                            threshold=0.5)

                result = pd.DataFrame({
                    'y_true': y_true,
                    'y_pred_proba': y_pred_proba
                })
                
                OUTPUT_FOREIGN_LANG_DATA_MULT = FOREIGN_LANG_DATA_MULT
                if EXPERIMENT_TYPE == 'A':
                    OUTPUT_FOREIGN_LANG_DATA_MULT = 0
                elif EXPERIMENT_TYPE == 'B':
                    OUTPUT_FOREIGN_LANG_DATA_MULT = -1
                
                OUTPUT_TOTAL_FOREIGN_DATA = int(TOTAL_DATA*FOREIGN_LANG_DATA_MULT)
                if EXPERIMENT_TYPE == 'A':
                    OUTPUT_TOTAL_FOREIGN_DATA = 0
                elif EXPERIMENT_TYPE == 'B':
                    OUTPUT_TOTAL_FOREIGN_DATA = TOTAL_DATA

                result_list.append({
                    'type':EXPERIMENT_TYPE,
                    'total_data':TOTAL_DATA, 
                    'seed':RANDOM_SEED,
                    'foreign_mult':OUTPUT_FOREIGN_LANG_DATA_MULT,
                    'total_foreign_data':OUTPUT_TOTAL_FOREIGN_DATA,
                    'f1':max_f1,
                    'recall':max_recall,
                    'precision':max_precision
                })

                os.remove("model.pt")

                if EXPERIMENT_TYPE != 'C':
                    break
                    
    final_result = pd.DataFrame(result_list) 
    final_result = final_result.sort_values(by=['total_data', 'foreign_mult'])
    final_result.to_csv('final_{}_{}_{}_result_combined_seed_{}.csv'.format(INDO_DATA_NAME,
                                                                           FOREIGN_DATA_NAME,
                                                                           MODEL_NAME,
                                                                           RANDOM_SEED),
                        index=False)
    
    result_list_final.extend(result_list)


############################
### START RANDOM_SEED 4 ###
############################

##### TOTAL_DATA 500 #####

~~Result_A_500~~
EarlyStopping! Epoch 48
Last: 35 	T-Loss: 0.616577 	T-Acc: 0.648889 	V-Loss: 0.654586 	V-Acc: 0.560000
## Test phase...
Final test Loss: 0.723768
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.33548387 	Recall: 1.00000000 	Prec: 0.50485437

~~Result_B_500~~
EarlyStopping! Epoch 30
Last: 17 	T-Loss: 0.684348 	T-Acc: 0.544444 	V-Loss: 0.691887 	V-Acc: 0.500000
## Test phase...
Final test Loss: 0.682702
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.44463246 	Recall: 0.95673077 	Prec: 0.52645503

~~Result_C_500_0.25~~
EarlyStopping! Epoch 49
Last: 36 	T-Loss: 0.623267 	T-Acc: 0.628774 	V-Loss: 0.635712 	V-Acc: 0.560000
## Test phase...
Final test Loss: 0.700349
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 74
Last: 61 	T-Loss: 0.461264 	T-Acc: 0.791468 	V-Loss: 0.406244 	V-Acc: 0.830601
## Test phase...
Final test Loss: 0.598166
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.67069857 	Recall: 0.59615385 	Prec: 0.70857143

~~Result_C_10981_0.5~~
EarlyStopping! Epoch 106
Last: 93 	T-Loss: 0.480636 	T-Acc: 0.778722 	V-Loss: 0.404861 	V-Acc: 0.824226
## Test phase...
Final test Loss: 0.590495
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.68161681 	Recall: 0.63942308 	Prec: 0.70370370

~~Result_C_10981_0.75~~
EarlyStopping! Epoch 76
Last: 63 	T-Loss: 0.495986 	T-Acc: 0.763733 	V-Loss: 0.404600 	V-Acc: 0.831512
## Test phase...
Final test Loss: 0.591029
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.67942279 	Recall: 0.64903846 	Prec: 0.69587629

~~Result_C_10981_1~~
EarlyStopping! Epoch 62
Last: 49 	T-Loss: 0

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 39
Last: 26 	T-Loss: 0.538588 	T-Acc: 0.726850 	V-Loss: 0.443408 	V-Acc: 0.806922
## Test phase...
Final test Loss: 0.589141
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.68766435 	Recall: 0.75480769 	Prec: 0.67094017

~~Result_C_10981_6~~
EarlyStopping! Epoch 50
Last: 37 	T-Loss: 0.537571 	T-Acc: 0.727942 	V-Loss: 0.445217 	V-Acc: 0.792350
## Test phase...
Final test Loss: 0.584273
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.69295195 	Recall: 0.75000000 	Prec: 0.67826087

~~Result_C_10981_7~~
EarlyStopping! Epoch 59
Last: 46 	T-Loss: 0.537688 	T-Acc: 0.726421 	V-Loss: 0.440910 	V-Acc: 0.795082
## Test phase...
Final test Loss: 0.583832
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.69182181 	Recall: 0.77403846 	Prec: 0.67083333

~~Result_C_10981_8~~
EarlyStopping! Epoch 45
Last: 32 	T-Loss: 0.53644

...

EarlyStopping! Epoch 60
Last: 47 	T-Loss: 0.534514 	T-Acc: 0.727052 	V-Loss: 0.433661 	V-Acc: 0.804189
## Test phase...
Final test Loss: 0.583056
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.69413512 	Recall: 0.77884615 	Prec: 0.67219917

~~Result_C_10981_6~~
EarlyStopping! Epoch 62
Last: 49 	T-Loss: 0.538254 	T-Acc: 0.725080 	V-Loss: 0.437956 	V-Acc: 0.812386
## Test phase...
Final test Loss: 0.583960
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.69742484 	Recall: 0.76442308 	Prec: 0.67948718

~~Result_C_10981_7~~
EarlyStopping! Epoch 78
Last: 65 	T-Loss: 0.538716 	T-Acc: 0.726395 	V-Loss: 0.440279 	V-Acc: 0.801457
## Test phase...
Final test Loss: 0.583678
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.68950682 	Recall: 0.76923077 	Prec: 0.66945607

~~Result_C_10981_8~~
EarlyStopping! Epoch 63
Last: 50 	T-Loss: 0.53742

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 91
Last: 78 	T-Loss: 0.466732 	T-Acc: 0.789282 	V-Loss: 0.408526 	V-Acc: 0.833333
## Test phase...
Final test Loss: 0.603311
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.66714433 	Recall: 0.62980769 	Prec: 0.68586387

~~Result_C_10981_0.5~~
EarlyStopping! Epoch 84
Last: 71 	T-Loss: 0.487583 	T-Acc: 0.772313 	V-Loss: 0.410104 	V-Acc: 0.834244
## Test phase...
Final test Loss: 0.599554
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.67456556 	Recall: 0.64423077 	Prec: 0.69072165

~~Result_C_10981_0.75~~
EarlyStopping! Epoch 82
Last: 69 	T-Loss: 0.497598 	T-Acc: 0.761304 	V-Loss: 0.403333 	V-Acc: 0.825137
## Test phase...
Final test Loss: 0.590106
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.67933962 	Recall: 0.64423077 	Prec: 0.69791667

~~Result_C_10981_1~~
EarlyStopping! Epoch 67
Last: 54 	T-Loss: 0.

/opt/conda/lib/python3.6/site-packages/torch/nn/modules/loss.py:498: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


EarlyStopping! Epoch 65
Last: 52 	T-Loss: 0.533401 	T-Acc: 0.730104 	V-Loss: 0.429156 	V-Acc: 0.809654
## Test phase...
Final test Loss: 0.581917
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.68136118 	Recall: 0.72115385 	Prec: 0.67264574

~~Result_C_10981_6~~
EarlyStopping! Epoch 57
Last: 44 	T-Loss: 0.536490 	T-Acc: 0.728448 	V-Loss: 0.437702 	V-Acc: 0.794171
## Test phase...
Final test Loss: 0.584212
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.69182181 	Recall: 0.77403846 	Prec: 0.67083333

~~Result_C_10981_7~~
EarlyStopping! Epoch 60
Last: 47 	T-Loss: 0.537646 	T-Acc: 0.726787 	V-Loss: 0.433161 	V-Acc: 0.806011
## Test phase...
Final test Loss: 0.581763
[Evaluate] Threshold argument set. Using 0.5 as threshold
[Evaluate] ##MAX## 
THRESHOLD: 0.500 	F1: 0.69581165 	Recall: 0.74038462 	Prec: 0.68444444

~~Result_C_10981_8~~
EarlyStopping! Epoch 51
Last: 38 	T-Loss: 0.53689

## Save

In [4]:
final_result = pd.DataFrame(result_list_final) 
final_result = final_result.sort_values(by=['total_data', 'foreign_mult'])
final_result.to_csv('final_{}_{}_{}_result_combined.csv'.format(INDO_DATA_NAME,
                                                               FOREIGN_DATA_NAME,
                                                               MODEL_NAME),
                        index=False)


In [5]:
print(final_result.shape)
final_result.head()

(288, 8)


,type,total_data,seed,foreign_mult,total_foreign_data,f1,recall,precision
1,B,500,4,-1.0,500,0.444632,0.956731,0.526455
97,B,500,5,-1.0,500,0.414686,0.975962,0.520513
193,B,500,6,-1.0,500,0.339806,0.995192,0.504878
0,A,500,4,0.0,0,0.335484,1.000000,0.504854
96,A,500,5,0.0,0,0.335484,1.000000,0.504854
